# Download Data

In [1]:
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import rasterio
import pandas as pd

sys.path.append(os.path.abspath(".."))  

In [2]:
from src.utils import load_config
from src.download_LAI_FAPAR import ProductDownloader

/Users/ivanapallister/WorkSpace/net-primary-production-1/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
config = load_config()

PRODUCTS = config["products"]
TILES = config["tiles"]
DATES = config["dates"]
SAVE_PATH = config["save_path"]

downloader = ProductDownloader(
    products=PRODUCTS,
    tiles=TILES,
    dates=DATES,
    save_path=SAVE_PATH,
)

downloader.download_all()


Directory already exists: ../data/LAI_FAPAR
Authenticated using refresh token.
Saved as ../data/LAI_FAPAR/LAI_T31UFT_2022-05-10.tiff
Failed: LAI - T31UFT - 2022-05-11 → [400] NoDataAvailable: There is no data available for the given extents. Could not find data for your load_collection request with catalog ID "urn:eop:VITO:TERRASCOPE_S2_LAI_V2". The catalog query had correlation ID "r-250729073529435186c120e39707a9dc" and returned 0 results. (ref: r-250729073529435186c120e39707a9dc)
Saved as ../data/LAI_FAPAR/LAI_T31UFT_2022-05-12.tiff
Failed: LAI - T31UFT - 2022-05-13 → [400] NoDataAvailable: There is no data available for the given extents. Could not find data for your load_collection request with catalog ID "urn:eop:VITO:TERRASCOPE_S2_LAI_V2". The catalog query had correlation ID "r-2507290735464b998b2e1307c33abc4e" and returned 0 results. (ref: r-2507290735464b998b2e1307c33abc4e)
Failed: LAI - T31UFT - 2022-05-14 → [400] NoDataAvailable: There is no data available for the given ext

# Compare Data?

In [3]:
vi_file = "./NPP_inputs/CLMS_products/VI_20220510T104621_S2A_T31UGT-010m_V101_LAI_FIXED_Qgis.tif"
openeo_file = "./FAPAR_T31UFT_2022-05-10.tiff"

In [4]:
# TODO: add open_data and vi_data
lai_open_decoded = np.where(open_data == 255, np.nan, open_data * 0.1)
vi_data_masked = np.where(vi_data == -32768, np.nan, vi_data)


NameError: name 'open_data' is not defined

In [5]:

with rasterio.open(vi_file) as src_vi, rasterio.open(openeo_file) as src_open:
    vi_data = src_vi.read(1).astype(float)
    open_data = src_open.read(1).astype(float)

    min_rows = min(vi_data.shape[0], open_data.shape[0])
    min_cols = min(vi_data.shape[1], open_data.shape[1])

    vi_data = vi_data[:min_rows, :min_cols]
    open_data = open_data[:min_rows, :min_cols]


diff = lai_open_decoded - vi_data_masked
diff_abs = np.abs(diff)

print(f"Mean abs difference: {np.nanmean(diff_abs):.2f}")
print(f"Max difference: {np.nanmax(diff_abs):.2f}")

# Assuming you've already created:
# - lai_open_decoded
# - vi_data_masked
# - diff = lai_open_decoded - vi_data_masked

plt.figure(figsize=(10, 5))
plt.hist(vi_data_masked[~np.isnan(vi_data_masked)].flatten(), bins=50, alpha=0.5, label='VI (float)')
plt.hist(lai_open_decoded[~np.isnan(lai_open_decoded)].flatten(), bins=50, alpha=0.5, label='OpenEO (scaled)')
plt.title("Value Distribution: LAI (May 10, 2022, T31UFT)")
plt.xlabel("LAI Value")
plt.ylabel("Frequency")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 8))
masked_diff = np.ma.masked_invalid(diff)
plt.imshow(masked_diff, cmap='bwr', vmin=-2, vmax=2)  # Clip to LAI difference ±2
plt.colorbar(label='LAI Difference (OpenEO - VI)')
plt.title("Spatial Difference: LAI (OpenEO - VI)")
plt.axis('off')
plt.tight_layout()
plt.show()

#TODO: Organize 

RasterioIOError: ./NPP_inputs/CLMS_products/VI_20220510T104621_S2A_T31UGT-010m_V101_LAI_FIXED_Qgis.tif: No such file or directory

# Input Preparation

In [12]:
import geopandas as gpd
from shapely.geometry import mapping, box
from rasterio.mask import mask
import rioxarray as rxr
import xarray as xr


In [4]:
provinces = gpd.read_file('../data/geo_boundaries/pv_2022.shp')

In [5]:
gelderland = provinces[provinces["PV_NAAM"].str.contains("Gelderland", case=False)]
gelderland_proj = gelderland.to_crs(epsg=32631) #reproject to UTM zone 31N
gelderland_shapes = gelderland_proj.geometry
gelderland_geojson = [mapping(shape) for shape in gelderland_shapes]

In [6]:
LAI_UGT_path = '../data/NPP_inputs/CLMS_products/VI_20220510T104621_S2A_T31UGT-010m_V101_LAI_FIXED_Qgis.tif'
FAPAR_UFT_path = '../data/NPP_inputs/CLMS_products/VI_20220510T104621_S2A_T31UFT-010m_V101_FAPAR_FIXED_Qgis.tif'
LAI_UFT_path = '../data/NPP_inputs/CLMS_products/VI_20220510T104621_S2A_T31UFT-010m_V101_LAI_FIXED_Qgis.tif'
FAPAR_UGT_path = '../data/NPP_inputs/CLMS_products/VI_20220510T104621_S2A_T31UGT-010m_V101_FAPAR_FIXED_Qgis.tif'

tile_files = {
    "LAI": [LAI_UGT_path, LAI_UFT_path],
    "FAPAR": [FAPAR_UGT_path, FAPAR_UFT_path]
}

In [7]:
def load_and_clip(path, geometry, crs_target="EPSG:32631", scale_factor=10000):
    raster = rxr.open_rasterio(path, masked=True).squeeze()
    raster_crs = raster.rio.crs

    if not isinstance(geometry, gpd.GeoDataFrame):
        gdf = gpd.GeoDataFrame(geometry=geometry, crs=crs_target)
    else:
        gdf = geometry.copy()

    if gdf.crs != raster_crs:
        gdf = gdf.to_crs(raster_crs)

    clipped = raster.rio.clip(gdf.geometry, raster_crs)
    clipped = clipped.rio.reproject(crs_target)
    clipped /= scale_factor
    return clipped

In [20]:
lai_layers = [load_and_clip(p, gelderland_shapes).sortby(["x", "y"]) for p in tile_files["LAI"]]
lai_combined = lai_layers[0].combine_first(lai_layers[1])
lai_combined.rio.to_raster("../data/NPP_processed_inputs/output_LAI_UTM31N.tif")

fapar_layers = [load_and_clip(p, gelderland_shapes).sortby("x").sortby("y") for p in tile_files["FAPAR"]]
fapar_combined = fapar_layers[0].combine_first(fapar_layers[1])
fapar_combined.rio.to_raster("../data/NPP_processed_inputs/output_FAPAR_UTM31N.tif")


In [8]:
temp_nc_path = '../data/NPP_inputs/Temps/single_level_2m_daily_mean_temp_2022_NL_ERA5.nc'
ssrd_nc_path = '../data/NPP_inputs/Temps/hourly_srrd_may_2022_netherlands.nc'

In [9]:
temp_out_path = '../data/NPP_processed_inputs/temperature_Gelderland.tif'
ssrd_out_path = '../data/NPP_processed_inputs/ssrd_Gelderland.tif'

In [ ]:
gelderland_wgs84 = gelderland.to_crs("EPSG:4326")
bbox = box(*gelderland_wgs84.total_bounds)
bbox_coords = {
    "lon_min": bbox.bounds[0],
    "lon_max": bbox.bounds[2],
    "lat_min": bbox.bounds[1],
    "lat_max": bbox.bounds[3]
}


def extract_temp_may(nc_path, varname, out_path):
    ds = xr.open_dataset(nc_path)
    ds_may = ds.sel(time=slice("2022-05-01", "2022-05-31"))
    da_mean = ds_may[varname].mean(dim="time")
    da_clip = da_mean.sel(
        latitude=slice(bbox_coords["lat_max"], bbox_coords["lat_min"]),  # descending!
        longitude=slice(bbox_coords["lon_min"], bbox_coords["lon_max"])
    )
    da_clip.rio.write_crs("EPSG:4326").rio.reproject("EPSG:32631").rio.to_raster(out_path)


def extract_ssrd_may(nc_path, varname, out_path):
    ds = xr.open_dataset(nc_path)
    if "valid_time" in ds.coords:
        ds = ds.rename({"valid_time": "time"})
    ssrd_raw = ds[varname]
    ssrd_may = ssrd_raw.sel(time=slice("2022-05-01", "2022-05-31"))
    ssrd_hourly = ssrd_may.diff("time")
    ssrd_daily = ssrd_hourly.resample(time="1D").sum()
    ssrd_mean = ssrd_daily.mean("time")
    ssrd_clip = ssrd_mean.sel(
        latitude=slice(bbox_coords["lat_max"], bbox_coords["lat_min"]),
        longitude=slice(bbox_coords["lon_min"], bbox_coords["lon_max"])
    )
    ssrd_clip.rio.write_crs("EPSG:4326").rio.reproject("EPSG:32631").rio.to_raster(out_path)
    print(f"Processed and saved: {out_path}")

In [11]:
extract_temp_may(temp_nc_path, "TEMP", temp_out_path)
extract_ssrd_may(ssrd_nc_path, "ssrd", ssrd_out_path)

✅ Processed and saved: ../data/NPP_processed_inputs/ssrd_Gelderland.tif


In [15]:
estk_full_path = '../data/NPP_inputs/ESTK_kaarten/NL_2022_10m.tif'
estk_clipped_path = '../data/NPP_processed_inputs/ESTK_Gelderland.tif'
estk_32631_path = '../data/NPP_processed_inputs/ESTK_Gelderland_32631.tif'

In [14]:
with rasterio.open(estk_full_path) as src:
    gelderland_reproj = gelderland.to_crs(src.crs)
    geometry = [mapping(g) for g in gelderland_reproj.geometry]

    clipped_array, clipped_transform = mask(src, geometry, crop=True)
    meta = src.meta.copy()
    meta.update({
        "height": clipped_array.shape[1],
        "width": clipped_array.shape[2],
        "transform": clipped_transform
    })

    with rasterio.open(estk_clipped_path, "w", **meta) as dest:
        dest.write(clipped_array)

In [17]:
estk_clip_rio = rxr.open_rasterio(estk_clipped_path, masked=True)
estk_clip_32631 = estk_clip_rio.rio.reproject("EPSG:32631")
estk_clip_32631.rio.to_raster(estk_32631_path)

In [ ]:
def check_raster(path, expected_crs=None):
    try:
        with rasterio.open(path) as src:
            size = os.path.getsize(path) / 1e6  
            crs = src.crs.to_string()
            bounds = src.bounds
            print(f"{os.path.basename(path)}")
            print(f"Size: {size:.1f} MB")
            print(f"CRS: {crs}")
            print(f"Bounds: {bounds}")
            if expected_crs and crs != expected_crs:
                print(f"CRS mismatch (expected {expected_crs})")
            print("")
    except Exception as e:
        print(f"Failed to read {path}: {e}")

In [ ]:
expected_crs = "EPSG:32631"

input_dir = "../data/NPP_processed_inputs/"
output_dir = "../NPP_outputs/"
files_to_check = [
    os.path.join(input_dir, "output_LAI_UTM31N.tif"), 
    os.path.join(input_dir, "output_FAPAR_UTM31N.tif"), 
    os.path.join(input_dir, "temperature_Gelderland.tif"),
    os.path.join(input_dir, "ssrd_Gelderland.tif"),
    os.path.join(input_dir, "ESTK_Gelderland_32631.tif")  # this one may still be EPSG:28992 #TODO: find out if that is true
]

for f in files_to_check:
    check_raster(f, expected_crs if "ESTK" not in f else None)

output_LAI_UTM31N.tif
Size: 333.6 MB
CRS: EPSG:32631
Bounds: BoundingBox(left=637310.0, bottom=5800020.0, right=763240.0, top=5733800.0)

output_FAPAR_UTM31N.tif
Size: 333.6 MB
CRS: EPSG:32631
Bounds: BoundingBox(left=637310.0, bottom=5800020.0, right=763240.0, top=5733800.0)

temperature_Gelderland.tif
Size: 0.0 MB
CRS: EPSG:32631
Bounds: BoundingBox(left=626913.5250109878, bottom=5719222.62178713, right=765030.0100936119, top=5837608.180429379)

ssrd_Gelderland.tif
Size: 0.0 MB
CRS: EPSG:32631
Bounds: BoundingBox(left=640609.5431240719, bottom=5736087.826350521, right=755546.7451127283, top=5831868.828007735)

ESTK_Gelderland_32631.tif
Size: 118.9 MB
CRS: EPSG:32631
Bounds: BoundingBox(left=634880.1402805636, bottom=5733224.9038243145, right=764152.7573115675, top=5825180.992101532)



# Calculate NPP

In [40]:
conversion_table_path = '../conversion_tables/25-04-15_conversietabel_MODIS_ESTK_LUTable_voorlopig.xlsx'
input_dir = '../data/NPP_processed_inputs/'
output_dir = '../NPP_outputs/'

In [32]:
conversion_table = pd.read_excel(conversion_table_path, engine="openpyxl")

In [33]:
conversion_table.columns = conversion_table.columns.str.strip() 
lut_dconversion_tablef = conversion_table.set_index("Pixelwaarde")
conversion_table.index = conversion_table.index.astype(int) 

conversion_table.head()


,Pixelwaarde,ESTK omschrijving,MODIS LC class,eps_max,SLA,Leaf_resp_base,Root_resp_base,Root_Leaf_ratio,Wood_resp_base,Wood_Leaf_ratio,T_max,T_min
0,1,Productiebos,6,0.000962,14.1,0.00604,0.00519,1.2,0.00397,0.182,8.31,-8
1,2,Akkerbouw_reg,1,0.001044,30.4,0.00980,0.00819,2.0,0.00000,0.000,12.02,-8
2,3,Ov_nat_grasland,2,0.000860,37.5,0.00980,0.00819,2.6,0.00000,0.000,12.02,-8
3,4,Groenvoorziening,7,0.001051,21.5,0.00778,0.00519,1.1,0.00371,0.203,9.50,-7
4,7,Infra_groen,4,0.001281,9.0,0.00869,0.00519,1.0,0.00436,0.079,8.61,-8


In [34]:
input_dir = '../data/NPP_processed_inputs/'

In [35]:
lai = rxr.open_rasterio(f"{input_dir}output_LAI_UTM31N.tif", masked=True).squeeze() / 10000
fapar = rxr.open_rasterio(f"{input_dir}output_FAPAR_UTM31N.tif", masked=True).squeeze() / 10000

In [36]:
temp = rxr.open_rasterio(f"{input_dir}temperature_Gelderland.tif", masked=True).squeeze()
temp = temp - 273.15

dssf = rxr.open_rasterio(f"{input_dir}ssrd_Gelderland.tif", masked=True).squeeze()

temp = temp.rio.reproject_match(lai)
dssf = dssf.rio.reproject_match(lai)
fapar = fapar.rio.reproject_match(lai)

print("Shapes (y, x):")
print("LAI:", lai.shape)
print("FAPAR:", fapar.shape)
print("TEMP:", temp.shape)
print("DSSF:", dssf.shape)

Shapes (y, x):
LAI: (6622, 12593)
FAPAR: (6622, 12593)
TEMP: (6622, 12593)
DSSF: (6622, 12593)


In [37]:
estk_path = f"{input_dir}ESTK_Gelderland_32631.tif"
estk_clip = rxr.open_rasterio(estk_path, masked=True).squeeze()
estk_clip_aligned = estk_clip.rio.reproject_match(lai)
unique_vals = np.unique(estk_clip_aligned.values[~np.isnan(estk_clip_aligned.values)])
print("ESTK classes found in raster:", unique_vals)

ESTK classes found in raster: [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 30. 31. 32. 33. 34. 37. 38. 44.
 45. 47. 48. 49. 50. 52.]


In [38]:
valid_classes = [int(v) for v in unique_vals if int(v) in conversion_table.index]
print(f"Will compute NPP for {len(valid_classes)} ecosystem classes.")

npp_array = xr.full_like(lai, np.nan)

for pixelwaarde in valid_classes:
    try:
        mask = estk_clip_aligned == pixelwaarde

        if mask.sum() == 0:
            print(f"Pixelwaarde {pixelwaarde} not found in aligned raster — skipping.")
            continue

        row = conversion_table.loc[pixelwaarde]
        eps_max = row["eps_max"]
        t_min = row["T_min"]
        t_max = row["T_max"]

        temp_masked = temp.where(mask)
        fapar_masked = fapar.where(mask)
        dssf_masked = dssf.where(mask)

        if np.isnan(temp_masked).all():
            print(f"All values masked out for class {pixelwaarde}, skipping.")
            continue

        temp_factor = ((temp_masked - t_min) * (t_max - temp_masked)) / ((t_max - t_min) ** 2)
        temp_factor = temp_factor.clip(min=0)

        gpp = eps_max * fapar_masked * dssf_masked
        npp = gpp * temp_factor

        npp_array = npp_array.where(~mask, npp)

    except Exception as e:
        print(f"Error processing {pixelwaarde}: {e}")

Will compute NPP for 29 ecosystem classes.


In [42]:
npp_out_path = f"{output_dir}NPP_Gelderland_May2022.tif"
npp_array.rio.write_crs(lai.rio.crs, inplace=True)
npp_array.rio.to_raster(npp_out_path)

# combine

In [ ]:
import geopandas as gpd
from shapely.geometry import mapping, box
from rasterio.mask import mask
import rioxarray as rxr
import xarray as xr

provinces = gpd.read_file('../data/geo_boundaries/pv_2022.shp')
gelderland = provinces[provinces["PV_NAAM"].str.contains("Gelderland", case=False)]
gelderland_proj = gelderland.to_crs(epsg=32631) #reproject to UTM zone 31N
gelderland_shapes = gelderland_proj.geometry
gelderland_geojson = [mapping(shape) for shape in gelderland_shapes]

LAI_UGT_path = '../data/NPP_inputs/CLMS_products/VI_20220510T104621_S2A_T31UGT-010m_V101_LAI_FIXED_Qgis.tif'
FAPAR_UFT_path = '../data/NPP_inputs/CLMS_products/VI_20220510T104621_S2A_T31UFT-010m_V101_FAPAR_FIXED_Qgis.tif'
LAI_UFT_path = '../data/NPP_inputs/CLMS_products/VI_20220510T104621_S2A_T31UFT-010m_V101_LAI_FIXED_Qgis.tif'
FAPAR_UGT_path = '../data/NPP_inputs/CLMS_products/VI_20220510T104621_S2A_T31UGT-010m_V101_FAPAR_FIXED_Qgis.tif'

tile_files = {
    "LAI": [LAI_UGT_path, LAI_UFT_path],
    "FAPAR": [FAPAR_UGT_path, FAPAR_UFT_path]
}

def load_and_clip(path, geometry, crs_target="EPSG:32631", scale_factor=10000):
    raster = rxr.open_rasterio(path, masked=True).squeeze()
    raster_crs = raster.rio.crs

    if not isinstance(geometry, gpd.GeoDataFrame):
        gdf = gpd.GeoDataFrame(geometry=geometry, crs=crs_target)
    else:
        gdf = geometry.copy()

    if gdf.crs != raster_crs:
        gdf = gdf.to_crs(raster_crs)

    clipped = raster.rio.clip(gdf.geometry, raster_crs)
    clipped = clipped.rio.reproject(crs_target)
    clipped /= scale_factor
    return clipped

lai_layers = [load_and_clip(p, gelderland_shapes).sortby(["x", "y"]) for p in tile_files["LAI"]]
lai_combined = lai_layers[0].combine_first(lai_layers[1])
lai_combined.rio.to_raster("../data/NPP_processed_inputs/output_LAI_UTM31N.tif")

fapar_layers = [load_and_clip(p, gelderland_shapes).sortby("x").sortby("y") for p in tile_files["FAPAR"]]
fapar_combined = fapar_layers[0].combine_first(fapar_layers[1])
fapar_combined.rio.to_raster("../data/NPP_processed_inputs/output_FAPAR_UTM31N.tif")

temp_nc_path = '../data/NPP_inputs/Temps/single_level_2m_daily_mean_temp_2022_NL_ERA5.nc'
ssrd_nc_path = '../data/NPP_inputs/Temps/hourly_srrd_may_2022_netherlands.nc'

temp_out_path = '../data/NPP_processed_inputs/temperature_Gelderland.tif'
ssrd_out_path = '../data/NPP_processed_inputs/ssrd_Gelderland.tif'

gelderland_wgs84 = gelderland.to_crs("EPSG:4326")
bbox = box(*gelderland_wgs84.total_bounds)
bbox_coords = {
    "lon_min": bbox.bounds[0],
    "lon_max": bbox.bounds[2],
    "lat_min": bbox.bounds[1],
    "lat_max": bbox.bounds[3]
}


def extract_temp_may(nc_path, varname, out_path):
    ds = xr.open_dataset(nc_path)
    ds_may = ds.sel(time=slice("2022-05-01", "2022-05-31"))
    da_mean = ds_may[varname].mean(dim="time")
    da_clip = da_mean.sel(
        latitude=slice(bbox_coords["lat_max"], bbox_coords["lat_min"]),  # descending!
        longitude=slice(bbox_coords["lon_min"], bbox_coords["lon_max"])
    )
    da_clip.rio.write_crs("EPSG:4326").rio.reproject("EPSG:32631").rio.to_raster(out_path)


def extract_ssrd_may(nc_path, varname, out_path):
    ds = xr.open_dataset(nc_path)
    if "valid_time" in ds.coords:
        ds = ds.rename({"valid_time": "time"})
    ssrd_raw = ds[varname]
    ssrd_may = ssrd_raw.sel(time=slice("2022-05-01", "2022-05-31"))
    ssrd_hourly = ssrd_may.diff("time")
    ssrd_daily = ssrd_hourly.resample(time="1D").sum()
    ssrd_mean = ssrd_daily.mean("time")
    ssrd_clip = ssrd_mean.sel(
        latitude=slice(bbox_coords["lat_max"], bbox_coords["lat_min"]),
        longitude=slice(bbox_coords["lon_min"], bbox_coords["lon_max"])
    )
    ssrd_clip.rio.write_crs("EPSG:4326").rio.reproject("EPSG:32631").rio.to_raster(out_path)
    print(f"Processed and saved: {out_path}")

extract_temp_may(temp_nc_path, "TEMP", temp_out_path)
extract_ssrd_may(ssrd_nc_path, "ssrd", ssrd_out_path)

estk_full_path = '../data/NPP_inputs/ESTK_kaarten/NL_2022_10m.tif'
estk_clipped_path = '../data/NPP_processed_inputs/ESTK_Gelderland.tif'
estk_32631_path = '../data/NPP_processed_inputs/ESTK_Gelderland_32631.tif'

with rasterio.open(estk_full_path) as src:
    gelderland_reproj = gelderland.to_crs(src.crs)
    geometry = [mapping(g) for g in gelderland_reproj.geometry]

    clipped_array, clipped_transform = mask(src, geometry, crop=True)
    meta = src.meta.copy()
    meta.update({
        "height": clipped_array.shape[1],
        "width": clipped_array.shape[2],
        "transform": clipped_transform
    })

    with rasterio.open(estk_clipped_path, "w", **meta) as dest:
        dest.write(clipped_array)

estk_clip_rio = rxr.open_rasterio(estk_clipped_path, masked=True)
estk_clip_32631 = estk_clip_rio.rio.reproject("EPSG:32631")
estk_clip_32631.rio.to_raster(estk_32631_path)

def check_raster(path, expected_crs=None):
    try:
        with rasterio.open(path) as src:
            size = os.path.getsize(path) / 1e6  
            crs = src.crs.to_string()
            bounds = src.bounds
            print(f"{os.path.basename(path)}")
            print(f"Size: {size:.1f} MB")
            print(f"CRS: {crs}")
            print(f"Bounds: {bounds}")
            if expected_crs and crs != expected_crs:
                print(f"CRS mismatch (expected {expected_crs})")
            print("")
    except Exception as e:
        print(f"Failed to read {path}: {e}")

expected_crs = "EPSG:32631"

input_dir = "../data/NPP_processed_inputs/"
output_dir = "../NPP_outputs/"
files_to_check = [
    os.path.join(input_dir, "output_LAI_UTM31N.tif"), 
    os.path.join(input_dir, "output_FAPAR_UTM31N.tif"), 
    os.path.join(input_dir, "temperature_Gelderland.tif"),
    os.path.join(input_dir, "ssrd_Gelderland.tif"),
    os.path.join(input_dir, "ESTK_Gelderland_32631.tif")  # this one may still be EPSG:28992 #TODO: find out if that is true
]

for f in files_to_check:
    check_raster(f, expected_crs if "ESTK" not in f else None)

conversion_table_path = '../conversion_tables/25-04-15_conversietabel_MODIS_ESTK_LUTable_voorlopig.xlsx'
input_dir = '../data/NPP_processed_inputs/'
output_dir = '../NPP_outputs/'

conversion_table = pd.read_excel(conversion_table_path, engine="openpyxl")

conversion_table.columns = conversion_table.columns.str.strip() 
lut_dconversion_tablef = conversion_table.set_index("Pixelwaarde")
conversion_table.index = conversion_table.index.astype(int) 

conversion_table.head()

input_dir = '../data/NPP_processed_inputs/'

lai = rxr.open_rasterio(f"{input_dir}output_LAI_UTM31N.tif", masked=True).squeeze() / 10000
fapar = rxr.open_rasterio(f"{input_dir}output_FAPAR_UTM31N.tif", masked=True).squeeze() / 10000

temp = rxr.open_rasterio(f"{input_dir}temperature_Gelderland.tif", masked=True).squeeze()
temp = temp - 273.15

dssf = rxr.open_rasterio(f"{input_dir}ssrd_Gelderland.tif", masked=True).squeeze()

temp = temp.rio.reproject_match(lai)
dssf = dssf.rio.reproject_match(lai)
fapar = fapar.rio.reproject_match(lai)

print("Shapes (y, x):")
print("LAI:", lai.shape)
print("FAPAR:", fapar.shape)
print("TEMP:", temp.shape)
print("DSSF:", dssf.shape)

estk_path = f"{input_dir}ESTK_Gelderland_32631.tif"
estk_clip = rxr.open_rasterio(estk_path, masked=True).squeeze()
estk_clip_aligned = estk_clip.rio.reproject_match(lai)
unique_vals = np.unique(estk_clip_aligned.values[~np.isnan(estk_clip_aligned.values)])
print("ESTK classes found in raster:", unique_vals)

valid_classes = [int(v) for v in unique_vals if int(v) in conversion_table.index]
print(f"Will compute NPP for {len(valid_classes)} ecosystem classes.")

npp_array = xr.full_like(lai, np.nan)

for pixelwaarde in valid_classes:
    try:
        mask = estk_clip_aligned == pixelwaarde

        if mask.sum() == 0:
            print(f"Pixelwaarde {pixelwaarde} not found in aligned raster — skipping.")
            continue

        row = conversion_table.loc[pixelwaarde]
        eps_max = row["eps_max"]
        t_min = row["T_min"]
        t_max = row["T_max"]

        temp_masked = temp.where(mask)
        fapar_masked = fapar.where(mask)
        dssf_masked = dssf.where(mask)

        if np.isnan(temp_masked).all():
            print(f"All values masked out for class {pixelwaarde}, skipping.")
            continue

        temp_factor = ((temp_masked - t_min) * (t_max - temp_masked)) / ((t_max - t_min) ** 2)
        temp_factor = temp_factor.clip(min=0)

        gpp = eps_max * fapar_masked * dssf_masked
        npp = gpp * temp_factor

        npp_array = npp_array.where(~mask, npp)

    except Exception as e:
        print(f"Error processing {pixelwaarde}: {e}")

npp_out_path = f"{output_dir}NPP_Gelderland_May2022.tif"
npp_array.rio.write_crs(lai.rio.crs, inplace=True)
npp_array.rio.to_raster(npp_out_path)